<a href="https://colab.research.google.com/github/ysut/weslink/blob/main/weslink_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use
Please click ▶ buttons.  

# Step 1.  
Let's upload your files.  
Excel file (.xlsx) is only available now (12/20/2022).

In [ ]:
from google.colab import files
uploaded = files.upload()

# Step 2.  
Get a path to your file from 📁. You can copy a path by right click.   
Then, please paste in the next form.

In [ ]:
input_file = input('Please enter a path: ')

# Step 3.
まとめてRUNしてください．

# RUN

In [22]:
import openpyxl as px
import pathlib

In [23]:
hgmd_link = '1'
ucsc_link = '2'
decipher_link = '3'
symbol_column = 'Gene.refGene'

## Code  
There is main code. You can modify it as needed.

In [24]:
class HyperLinks:
    def __init__(self, input_file, hgmd_link, ucsc_link, decipher_link, symbol_column):
        self.input_file = input_file
        self.workbook = px.load_workbook(self.input_file)
        self.sheet_names = self.workbook.sheetnames
        self.sheets = self.workbook.worksheets
        self.symbol_column = symbol_column
        self.insert_column_hgmd = px.utils.cell.get_column_letter(int(hgmd_link))
        self.insert_column_ucsc = px.utils.cell.get_column_letter(int(ucsc_link))
        self.insert_column_decipher = px.utils.cell.get_column_letter(int(decipher_link))
        self.input_path = pathlib.Path(self.input_file)
        self.input_file_name = self.input_path.stem
    
    def __del__(self):
        self.workbook.close()
        
    def _search_column_letter(self, keywords):
        result = {}
        for i, sheet in enumerate(self.sheets):
            for cell in sheet['1']:
                try:
                    value = str(cell.value)
                except: 
                    continue
                for keyword in keywords:
                    if value == keyword:
                        letter = cell.column_letter
                        result[self.sheet_names[i]] = letter
                        break
                    else:
                        pass
        return result 
    
    def hyperlink_hgmd(self):
        symbol_column_letters = self._search_column_letter([self.symbol_column])
        for sheet_num, sheet in enumerate(self.sheets):
            symbol_column_letter = symbol_column_letters[self.sheet_names[sheet_num]]
            max_row = sheet.max_row
            for index in range(max_row):
                gene_symbol = sheet[f'{symbol_column_letter}{index + 1}'].value             
                hgmd_url = f'https://my.qiagendigitalinsights.com/bbp/view/hgmd/pro/gene.php?gene={gene_symbol}'
                sheet[f'{self.insert_column_hgmd}{index + 1}'].hyperlink = hgmd_url 

    def save_dl(self):
        output_name = f'{self.input_file}.weslinked.xlsx'
        self.workbook.save(output_name)
        files.download(output_name)

def main():
    linkanno = HyperLinks(input_file, hgmd_link, ucsc_link, decipher_link, symbol_column)
    linkanno.hyperlink_hgmd()
    linkanno.save_dl()
    del linkanno


In [ ]:
main()

# Step 4.  
You can delete uploaded files.

In [26]:
!find ./ -type f -name "*.xlsx" | xargs rm -rf